#### Here I will document my method for cleaning my data files, which were taken from Kaggle's dataset “Comments on articles published in the New York Times” (https://www.kaggle.com/aashita/nyt-comments).
    
This code is broken into two sections - one cleans the data files with articles and one cleans the data files with comments.

The first block of this code combines my data files into a single large file, one each for articles and comments.

In [2]:
import glob
import os
os.chdir("../Data")

import pandas as pd
import glob

interesting_files = glob.glob("../Comments*.csv")
df_list = []

if len(df_list) > 0: 

    for filename in sorted(interesting_files):
        df_list.append(pd.read_csv(filename))
    full_df = pd.concat(df_list)

    full_df.to_csv('allComments.csv', index=False)

interesting_files = glob.glob("Articles*.csv")
df_list = []

if len(df_list) > 0: 
    for filename in sorted(interesting_files):
        df_list.append(pd.read_csv(filename))
    full_df = pd.concat(df_list)

    full_df.to_csv('allArticles.csv', index=False)

# Initial goals: 

-Make sure the contents of each field are the correct type and have no missing data (i.e. scrub the 'NaN' from the 'abstract' field)

-Make sure that the data comes properly tokenized

-Convert all words to lowercase (to avoid confusion between uppercase and lowercase versions of the same word)

###### From the above code, the only integer columns are 2, 7 and 9. The rest are string columns and need to be converted to lowercase. 

###### I want to check for cells that have missing elements.

###### Now we want a separate section to clean the comment files.

In [7]:
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')



#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allComments.csv")

train = pd.read_csv("allComments.csv")

#train['recommendations'].head(5)
train

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1.491245e+09,This project makes me happy to be a 30+ year T...,22022598.0,22022598.0,<br/>,comment,1.491237e+09,1.0,False,0.0,...,"Riverside, CA",NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
1,1.491189e+09,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350.0,NaN,comment,1.491180e+09,1.0,False,0.0,...,<br/>,NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
2,1.491189e+09,Brilliant work from conception to execution. I...,22017334.0,22017334.0,<br/>,comment,1.491179e+09,1.0,False,0.0,...,Raleigh NC,NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
3,1.491168e+09,NYT reporters should provide a contributor's l...,22015913.0,22015913.0,<br/>,comment,1.491150e+09,1.0,False,0.0,...,"Missouri, USA",NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
4,1.491168e+09,Could only have been done in print. Stunning.,22015466.0,22015466.0,<br/>,comment,1.491147e+09,1.0,False,0.0,...,"Tucson, Arizona",NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
5,1.491143e+09,Thank you New York Times. People should be sup...,22012085.0,22012085.0,<br/>,comment,1.491129e+09,1.0,False,0.0,...,new york,NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
6,1.491061e+09,Proof that photojournalism is alive and well. ...,22003784.0,22003784.0,<br/>,comment,1.491056e+09,1.0,False,0.0,...,"East Northport, NY",NaN,NaN,0.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
7,1.491252e+09,"The OASIS Initiative, which I started with Pro...",22024897.0,22024897.0,<br/>,userReply,1.491248e+09,2.0,False,22015913.0,...,"Tahoma, CA",NaN,NaN,22015913.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
8,1.491669e+09,I agree. I've just spent 30 minutes trying to ...,22082978.0,22082978.0,NaN,userReply,1.491665e+09,2.0,False,22015913.0,...,Ann Arbor,NaN,NaN,22015913.0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2.0,News
9,1.491064e+09,How about Katrina Pierson? Back to Palookavill...,22004930.0,22004930.0,<br/>,comment,1.491061e+09,1.0,False,0.0,...,California,NaN,NaN,0.0,58def3237c459f24986d7c84,Unknown,OpEd,823.0,23.0,Op-Ed


In [8]:
train= train.astype(str)
train.fillna(0)
strings = [1, 5, 10, 22, 24, 25, 26, 29, 30, 33]

wn = nltk.WordNetLemmatizer()

def lemmatize_text(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in strings:         
            doc[column] = doc[column].str.replace('[^\w\s]','')
            doc[column] = doc[column].str.lower()
            #doc[column] = doc[column].str.strip()
            doc[column] = doc[column].replace(np.nan, '', regex=True)
            doc[column].apply(nltk.word_tokenize)
            doc[column].apply(lemmatize_text)
            doc[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        else:
            doc[column] = doc[column].astype(str)
            continue
    return doc

clean_comments = clean_articles(train)
#The second command takes awhile to run
clean_comments.head(5)

clean_comments['recommendations'].head(50)

0       2.0
1       1.0
2       3.0
3       7.0
4       5.0
5       8.0
6       9.0
7       1.0
8       0.0
9      60.0
10     96.0
11     61.0
12    154.0
13    178.0
14    134.0
15     91.0
16    134.0
17     33.0
18     57.0
19    128.0
20    168.0
21    111.0
22     44.0
23     75.0
24    137.0
25    151.0
26     60.0
27    104.0
28     15.0
29     28.0
30    107.0
31     43.0
32     34.0
33     49.0
34     18.0
35     42.0
36     60.0
37     89.0
38     23.0
39     38.0
40     79.0
41     37.0
42     89.0
43     21.0
44     72.0
45     79.0
46     20.0
47     15.0
48     39.0
49     29.0
Name: recommendations, dtype: object

In [9]:
print("Any null values left: "), print(clean_comments.isnull().values.any())

Any null values left: 
False


(None, None)

In [10]:
print(len(clean_comments))
clean_comments.nunique()

237227


approveDate              2730
commentBody              4997
commentID                5000
commentSequence          5000
commentTitle                2
commentType                 3
createDate               4929
depth                       3
editorsSelection            3
parentID                  817
parentUserDisplayName     723
permID                   5000
picURL                    495
recommendations           269
recommendedFlag             1
replyCount                 23
reportAbuseFlag             1
sharing                     3
status                      2
timespeople                 2
trusted                     3
updateDate               2861
userDisplayName          3029
userID                   3578
userLocation             1512
userTitle                   1
userURL                     1
inReplyTo                 817
articleID                  14
sectionName                 7
newDesk                    11
articleWordCount           14
printPage                   7
typeOfMate

I see that there are no null values remaining, but looking at the dataframe I see that several columns contain nothing but 'nan' strings or otherwise have only one value. I want to drop the commentTitle (contains only <br/> or nan), recommendedFlag, reportAbuseFlag, status, timespeople, userTitle and userURL columns.

In [11]:
clean_comments.drop(columns=['commentTitle', 'recommendedFlag', 'reportAbuseFlag', 'status', 'timespeople', 'userTitle', 'userURL'], axis=1, inplace=True)

clean_comments.head(5)

,approveDate,commentBody,commentID,commentSequence,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,...,userDisplayName,userID,userLocation,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186.0,this project makes me happy to be a 30 year ti...,22022598.0,22022598.0,comment,1491237056.0,1.0,False,0.0,nan,...,rob gayle,46006296.0,riverside ca,0.0,58def1347c459f24986d7c80,unknown,insider,716.0,2.0,news
1,1491188619.0,stunning photos and reportage infuriating that...,22017350.0,22017350.0,comment,1491180489.0,1.0,False,0.0,nan,...,susan a,29202761.0,br,0.0,58def1347c459f24986d7c80,unknown,insider,716.0,2.0,news
2,1491188617.0,brilliant work from conception to execution iv...,22017334.0,22017334.0,comment,1491179470.0,1.0,False,0.0,nan,...,meta,63944806.0,raleigh nc,0.0,58def1347c459f24986d7c80,unknown,insider,716.0,2.0,news
3,1491167820.0,nyt reporters should provide a contributors li...,22015913.0,22015913.0,comment,1491150196.0,1.0,False,0.0,nan,...,tom wyrick,1266184.0,missouri usa,0.0,58def1347c459f24986d7c80,unknown,insider,716.0,2.0,news
4,1491167815.0,could only have been done in print stunning,22015466.0,22015466.0,comment,1491147284.0,1.0,False,0.0,nan,...,joe sharkey,61121360.0,tucson arizona,0.0,58def1347c459f24986d7c80,unknown,insider,716.0,2.0,news


###### I check my working directory to make sure I am saving the files where I want them stored.

In [12]:
com_file_name = "cleaned_comment_data.csv"
clean_com_csv = clean_comments.to_csv(com_file_name, encoding='utf-8', index=False)

In [14]:
nbconvert to script "Data cleaning.py"

SyntaxError: invalid syntax (<ipython-input-14-42217beaa64d>, line 1)